In [6]:
# 1
# # current_working_directory = '/drive/MyDrive/Colab Notebooks/好玩rag_makeup/'  # Replace with your project name
# current_working_directory = '/drive/MyDrive/Colab Notebooks/好玩rag_makeup/'  # Replace with your project name
# 
# import os
# def check_if_colab_environment():
#     try:
#         import google.colab
#         return True
#     except ImportError:
#         return False
# 
# 
# if check_if_colab_environment() is True:
#     from google.colab import drive
#     drive.mount('/drive')
# 
# 
# if current_working_directory.endswith('/'):
#     current_working_directory = current_working_directory[:-1]
# 
# os.chdir(current_working_directory)


In [7]:
current_dir = os.getcwd()
current_dir

'/home/arden/Documents/好玩rag_makeup'

In [8]:
# environment installation
# !pip install chromadb
!pip install pysqlite3-binary
!pip install sqlite3 >= 3.35.0
!pip install langchain langchain_chroma langchain_community langchainhub langchain-openai
!pip install jq
!pip install -qU langchain-milvus
# !pip install langchain-openai
# !pip uninstall langchain-openai -y
# !pip install langchain-openai

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
DEPRECATION: distro-info 0.23ubuntu1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: python-debian 0.1.36ubuntu1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3.10 -m pip install --upgrade pip
Defaulting to user

In [9]:
!pip install sqlite3 >= 3.35.0

ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [10]:
from langchain_community.document_loaders import JSONLoader
import json
from typing import AsyncIterator, Iterator

from langchain_core.document_loaders import BaseLoader
from langchain_core.documents import Document


class PTTDocumentLoader(BaseLoader):
    def __init__(self, file_path: str) -> None:
        self.file_path = file_path

    def lazy_load(self) -> Iterator[Document]:  # <-- Does not take any arguments

        with open(self.file_path, 'r') as f:
            makeup = json.load(f)

            line_number = 0
            for line in makeup:
                yield Document(
                    page_content=line,
                    metadata={"line_number": line_number, "source": self.file_path},
                )
                line_number += 1

    # alazy_load is OPTIONAL. If you leave out the implementation, a default implementation which delegates to lazy_load will be used!
    async def alazy_load(self) -> AsyncIterator[Document]:  # <-- Does not take any arguments
        async with aiofiles.open(self.file_path, 'r', encoding='utf-8') as f:
            contents = await f.read()
            makeup = json.loads(contents)

            line_number = 0
            for line in makeup:
                yield Document(
                    page_content=line,
                    metadata={"line_number": line_number, "source": self.file_path},
                )
                line_number += 1

all_splits = PTTDocumentLoader("data/json/posts_chunk_防曬.json")



In [11]:

# Load the documents from the loader
loaded_documents = all_splits.load()

# Now you can access the loaded documents as a list
first_document = loaded_documents[0]

# Print the content of the first document
print(first_document.page_content)

心得: Dr.Althea 女神水光氣墊粉餅使用心得
使用者評價:
1. 眼睛好美喲
2. 剛完妝那張真是美翻了
3. 盒子好漂亮
4. 我覺得浮粉蠻嚴重的lol 不是會和出油融合的感覺
5. 近照感覺不太ok
6. 近照全臉都浮粉了吧 不過剛上很美
品牌:sofina 面膜 hello out you
標籤:液 粉底液 粉餅 蜜粉 遮瑕膏 粉撲 粉盒 面膜 韓國 台灣 遮瑕 自然 不悶 防曬 隔離 細紋 底妝 妝前 裸妝 眼下細紋 粉底 氣墊粉餅 敏感 補妝 控油 光澤感


DEPRECATION: distro-info 0.23ubuntu1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: python-debian 0.1.36ubuntu1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [14]:
from langchain_openai import OpenAIEmbeddings
# vectorstore = Chroma.from_documents(documents=loaded_documents, embedding=OpenAIEmbeddings())
from langchain_milvus import Milvus
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")


# The easiest way is to use Milvus Lite where everything is stored in a local file.
# If you have a Milvus server you can use the server URI such as "http://localhost:19530".
URI = "./milvus_example.db"

vectorstore = Milvus(
    embedding_function=embeddings,
    connection_args={"uri": URI},

)

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})
retrieved_docs = retriever.invoke("含防曬 不黏膩  不厚重 的化妝品推薦")

len(retrieved_docs)

0

In [15]:
print(retrieved_docs[1].page_content)

IndexError: list index out of range

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("男性香水品牌以及他們的特色 "):
    print(chunk, end="", flush=True)